In [1]:
from unityagents import UnityEnvironment
import numpy as np

from models import Actor, Critic
from DDPG import DDPG_Agent
from tools import PlotTool

from unityagents import UnityEnvironment
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device, torch.cuda.is_available())

from ipywidgets import *
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib notebook

cuda:0 True


In [2]:
env         = UnityEnvironment(file_name="../Tennis_Linux/Tennis.x86_64", no_graphics=False)
brain_name  = env.brain_names[0]
brain       = env.brains[brain_name]
env_info    = env.reset(train_mode=True)[brain_name]
num_agents  = len(env_info.agents)
action_size = brain.vector_action_space_size
state_size  = len(env_info.vector_observations[0])
print("state size:",state_size, "action size:", action_size)
print("state shape: ", env_info.vector_observations.shape)

DEBUG:unityagents:The true file name is Tennis
DEBUG:unityagents:This is the launch string /home/yousof/AI/courses/deep_RL_udacity/projects/DRLND_Collaboration_Competition_P3/../Tennis_Linux/Tennis.x86_64
INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


state size: 24 action size: 2
state shape:  (2, 24)


In [3]:
agent = DDPG_Agent(state_size=state_size, 
                   action_size=action_size, 
                   actor_model=Actor,
                   critic_model=Critic,
                   device=device,
                   num_agents= num_agents, 
                   seed=1,
                   tau=3e-1,
                   batch_size=2048,
                   discount_factor = 0.99,
                   actor_learning_rate=1e-4,
                   critic_learning_rate=1e-3)

models_dic = {agent.actor_local: 'checkpoint_actor.pth',
              agent.critic_local: 'checkpoint_critic.pth',
              agent.critic2_local: 'checkpoint_critic2.pth',
              agent.actor_opt: 'checkpoint_actor_opt.pth',
              agent.critic_opt: 'checkpoint_critic_opt.pth',
              agent.critic2_opt: 'checkpoint_critic2_opt.pth'}


for m in models_dic:
    m.load_state_dict(torch.load(models_dic[m]))
    try:
        m.eval()
    except:
        pass

FileNotFoundError: [Errno 2] No such file or directory: 'checkpoint_critic2.pth'

In [5]:
env_info = env.reset(train_mode=False)[brain_name] # reset the environment
states = env_info.vector_observations              # get the current state
score = 0
step = 0

In [6]:
%matplotlib notebook
rewardPlot = PlotTool(number_of_lines=num_agents, desc = "R" )

#env_info = env.reset(train_mode=False)[brain_name] # reset the environment
states = env_info.vector_observations              # get the current state
score = 0
step = 0


while True:
    _, actions_without_noise  = agent.act(states,add_noise=False)
    actions = np.clip(actions_without_noise, -1, 1)
    env_info = env.step(actions)[brain_name]
    states = env_info.vector_observations
    reward = env_info.rewards
    score = score + np.array(reward)
    rewardPlot.push_date(score.reshape(1, num_agents))
    step += 1
    if step % 20 == 0:
        rewardPlot.draw()
    #if step == 21:
    #    input("Press Enter to continue...")
    if np.any(env_info.local_done):
        break
rewardPlot.draw()

<IPython.core.display.Javascript object>

In [16]:
env.close()